In [47]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1, model="gpt-4o-mini")

memory = ConversationSummaryMemory(llm=chat)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi my name is Lee and I love to run.", "Nice to meet you Mr. Lee. It's great to hear you like running!")
add_message("And My grandpa is awesome human being.", "Oh! I see. Your grandpa is great person.")

get_history()

{'history': "The human, named Lee, introduces himself and shares his love for running. The AI responds positively, expressing pleasure in hearing about Lee's interest in running. Lee also mentions that his grandpa is an awesome human being, to which the AI acknowledges and agrees that Lee's grandpa is a great person."}